## $P^{n}(X_{4})$ の位数

$$
P^{n}(X_{4}) = \{(\sigma_{1}, \dots, \sigma_{n}) | \sigma_{1}\sigma_{2} \cdots \sigma_{n} = 1, \sigma_{i} \neq \sigma_{j} [\forall i,j] \}
$$

In [1]:
require 'benchmark'
nil

## Cyclic permutations

In [ ]:
def act(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else 
    return sigma[(sigma.index(k)+1)%4]
  end
end; nil

In [ ]:
def act01(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else
    ind = sigma.index(k)+1
    return sigma[ind].then{|x| (x.nil?) ? sigma[0] : x}
  end
end; nil

In [ ]:
def act02(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else
    ind = (sigma.index(k)+1).then{|x| (x > 4) ? 0 : x}
    return sigma[ind]
  end
end; nil

In [ ]:
def act03(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else
    ind = (sigma.index(k)+1).then{|x| (x == 4) ? 0 : x}
    return sigma[ind]
  end
end; nil

In [ ]:
def act04(sigma, k)
  if k.is_a? Array
    k.map{|v| act(sigma, v)}
  else
    sigma[(sigma.index(k)+1).then{|x| (x == 4) ? 0 : x}]
  end
end; nil

In [ ]:
marr = (0..3).to_a

marr.permutation.each do |sigma|
  p sigma if act(sigma, marr) != act04(sigma, marr)
end; nil

In [ ]:
count = (10**6)*2

Benchmark.bm(10) do |x|
#   x.report "act" do
#     myarr = [0,1,2,3]
#     count.times do |k|
#       myarr = act([1,3,0,2], myarr)
#     end
#   end
  x.report "act01" do
    myarr = [0,1,2,3]
    count.times do |k|
      myarr = act01([1,3,0,2], myarr)
    end
  end
  x.report "act03" do
    myarr = [0,1,2,3]
    count.times do |k|
      myarr = act03([1,3,0,2], myarr)
    end
  end
  x.report "act04" do
    myarr = [0,1,2,3]
    count.times do |k|
      myarr = act04([1,3,0,2], myarr)
    end
  end
end; nil

In [ ]:
a = (0..3).to_a
sigma = [2,3,0,1]
tau = [3,2,0,1]
sa = act(sigma, a)
tsa = act(tau, sa)

puts "#{a}   |-#{sigma}->   #{sa}   |-#{tau}->   #{tsa}"
nil

## Tuples of cyclic permutations

In [ ]:
k = 4
CPLIST = (0..k-2).to_a.permutation.map{|s| s << (k-1)}
INDICES = (0..CPLIST.size-1).to_a

INITIAL = (0..k-1).to_a
EVENS = (1..100).map{|k| 2*k}

CPLIST

In [ ]:
n = 4
cp_tuples = [()]
n.times{|i| cp_tuples = cp_tuples.product(CPLIST)}

puts "#{cp_tuples[3]} \n---"

cp_tuples.map! do |cps|
  n.times.map{|k| (cps.pop).tap{|x| cps.flatten!(1)}}
end

p cp_tuples[3]

nil

In [ ]:
n = 4

cp_tuples = INDICES
(n-1).times{|i| cp_tuples = cp_tuples.product(INDICES)}

puts "#{cp_tuples[10].flatten} \n---"

cp_tuples.map!{|cps| cps.flatten.map{|i| CPLIST[i]}}

p cp_tuples[10]

nil

In [ ]:
n = 4

cp_tuples = INDICES
(n-1).times{|i| cp_tuples = cp_tuples.product(INDICES)}
cp_tuples
puts "#{cp_tuples[10].flatten} \n---"

cp_tuples.map!{|cps| cps.flatten.map{|i| CPLIST[i]}}

p cp_tuples[10]

nil

In [ ]:
initial = (0..k-1).to_a

p cp_tuples.size

cp_tuples.delete_if do |cps|
  marr = initial.dup
  cps.each{|sigma| marr = act(sigma, marr)}
  (marr != (0..k-1).to_a) || (cps.uniq.size == 1)
end

p cp_tuples.size

cp_tuples.each do |cps|
  marr = INITIAL.dup
  following = cps.map do |sigma|
    " |--> #{act(sigma, marr)}"
  end.join
  p initial.to_s + following
end
nil

## Getting the sequence

In [ ]:
def get_cardinality(num)
  cp_tuples_inds = INDICES
  (num - 1).times{|i| cp_tuples_inds = cp_tuples_inds.product(INDICES)}
  
  cp_tuples_inds.map do |cps|
    marr = INITIAL.dup
    (cps.flatten).each{|i| marr = act02(CPLIST[i], marr)}
    (marr == INITIAL)
  end.count(true)
end; nil

In [ ]:
get_cardinality(8)

In [ ]:
max = 4
EVENS[0..max-1].map{|even| get_cardinality(even)}

In [ ]:
class Tree
  def initialize(cp)
    @cp = cp
    @children = []
  end
  attr_accessor :cp, :children
end; nil

In [ ]:
root = Tree.new(0)
CPLIST.each{|cp| root.children << cp}
root.children

## Experiments

In [ ]:
def get_cardinality_01(num)
  cp_tuples = [()]
  num.times{|i| cp_tuples = cp_tuples.product(CPLIST)}
  cp_tuples.map!{|cps| num.times.map{|k| (cps.pop).tap{|x| cps.flatten!(1)}}}
  
  cp_tuples.delete_if do |cps|
    marr = INITIAL.dup
    cps.each{|sigma| marr = act(sigma, marr)}
    (marr != INITIAL) #|| (cps.uniq.size == 1)
  end
  return cp_tuples.size
end; nil

In [ ]:
def get_cardinality_02(num)
  cp_tuples = [()]
  num.times{|i| cp_tuples = cp_tuples.product(CPLIST)}
  cp_tuples.map!{|cps| num.times.map{|k| (cps.pop).tap{|x| cps.flatten!(1)}}}
    
  cp_tuples.map do |cps|
    marr = INITIAL.dup
    cps.each{|sigma| marr = act(sigma, marr)}
    (marr == INITIAL) #|| (cps.uniq.size == 1)
  end.count(true)
end; nil

In [ ]:
get_cardinality_02(6)

In [ ]:
Benchmark.bm(10) do |x|
  n = 6
  x.report("get_card") { get_cardinality(n) }
  x.report("get_card_01") { get_cardinality_01(n) }
  x.report("get_card_02") { get_cardinality_02(n) }
end; nil

In [ ]:
Benchmark.bm(15) do |x|
  num = 6
  x.report("get_card") { get_cardinality(num) }
  x.report("get_card act02") do
    cp_tuples_inds = INDICES
    (num - 1).times{|i| cp_tuples_inds = cp_tuples_inds.product(INDICES)}
  
    cp_tuples_inds.map do |cps|
      marr = INITIAL.dup
      (cps.flatten).each{|i| marr = act02(CPLIST[i], marr)}
      (marr == INITIAL)
    end.count(true)
  end
  x.report("get_card fix") do
    cp_tuples_inds = INDICES
    (num - 1).times{|i| cp_tuples_inds = cp_tuples_inds.product(INDICES)}
  
    cp_tuples_inds.map do |cps|
      cps.flatten!
      marr = INITIAL.dup
      cps.each{|i| marr = act02(CPLIST[i], marr)}
      (marr == INITIAL)
    end.count(true)
  end
end; nil